In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
MAPBOX_TOKEN = os.getenv("MAPBOX_TOKEN")


In [4]:
import pandas as pd

test_df = pd.read_csv("../data/raw/test.csv")

print(test_df.shape)
test_df.head()


(5404, 20)


,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,2591820310,20141006T000000,4,2.25,2070,8893,2.0,0,0,4,8,2070,0,1986,0,98058,47.4388,-122.162,2390,7700
1,7974200820,20140821T000000,5,3.00,2900,6730,1.0,0,0,5,8,1830,1070,1977,0,98115,47.6784,-122.285,2370,6283
2,7701450110,20140815T000000,4,2.50,3770,10893,2.0,0,2,3,11,3770,0,1997,0,98006,47.5646,-122.129,3710,9685
3,9522300010,20150331T000000,3,3.50,4560,14608,2.0,0,2,3,12,4560,0,1990,0,98034,47.6995,-122.228,4050,14226
4,9510861140,20140714T000000,3,2.50,2550,5376,2.0,0,0,3,9,2550,0,2004,0,98052,47.6647,-122.083,2250,4050


In [5]:
import requests
from pathlib import Path

TEST_IMG_DIR = Path("../data/images/test")
TEST_IMG_DIR.mkdir(parents=True, exist_ok=True)

def download_satellite_image(property_id, lat, lon, zoom=18, size=256):
    url = (
        f"https://api.mapbox.com/styles/v1/mapbox/satellite-v9/static/"
        f"{lon},{lat},{zoom}/"
        f"{size}x{size}"
        f"?access_token={MAPBOX_TOKEN}"
    )

    save_path = TEST_IMG_DIR / f"{property_id}.png"

    if save_path.exists():
        return True  # already downloaded

    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            with open(save_path, "wb") as f:
                f.write(response.content)
            return True
        else:
            return False
    except:
        return False


In [6]:
from tqdm import tqdm

success, failed = 0, 0

for _, row in tqdm(test_df.iterrows(), total=len(test_df)):
    ok = download_satellite_image(
        property_id=row["id"],
        lat=row["lat"],
        lon=row["long"]
    )

    if ok:
        success += 1
    else:
        failed += 1

print("Downloaded:", success)
print("Failed:", failed)


100%|█████████████████████████████████████| 5404/5404 [1:48:44<00:00,  1.21s/it]

Downloaded: 5404
Failed: 0


In [8]:
import os

print("Images in test folder:", len(os.listdir(TEST_IMG_DIR)))
print("Rows in test data:", len(test_df))


Images in test folder: 5396
Rows in test data: 5404
